In [1]:
import torch
import hiera
from torchvision.transforms import transforms
from torchvision.transforms.functional import InterpolationMode
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
import skimage
from skimage import io
from skimage import exposure
from skimage import segmentation
from scipy.ndimage import gaussian_filter
from scipy import ndimage as ndi
from skimage.morphology import reconstruction
from skimage.filters import sobel
from skimage.transform import rescale
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD


import sys
import glob

/home/axd497/.usr/local/python/3.10.4/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
os.chdir('/mnt/pan/SOM_CCCC_JGS25/durmaza/organoid_images/')

In [3]:
# Load Hiera Model
model = hiera.hiera_base_224(pretrained=True, checkpoint="mae_in1k")
transform_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

In [4]:
f_list = glob.glob('3d gastruloids all images/MR/24H/*tif')
mr24 = []
for f in f_list:
    temp=io.imread(f, as_gray=False, plugin='tifffile').astype(np.float32)
    im = transform_norm(temp/255.0)
    output, intermediates = model(im[None,], return_intermediates=True)
    mr24.append(intermediates[3][0,].detach().numpy())

mr24 = np.stack(mr24, axis=0)

In [5]:
f_list = glob.glob('good_batch/MR/*tif')
mr72 = []
for f in f_list:
    temp=io.imread(f, as_gray=False, plugin='tifffile').astype(np.float32)
    if temp.ndim < 3: continue
    im = transform_norm(temp/255.0)
    output, intermediates = model(im[None,], return_intermediates=True)
    mr72.append(intermediates[3][0,].detach().numpy())

mr72 = np.stack(mr72, axis=0)

In [6]:
f_list = glob.glob('3d gastruloids all images/GD/24H/*tif')
gd24 = []
for f in f_list:
    temp=io.imread(f, as_gray=False, plugin='tifffile').astype(np.float32)
    if temp.ndim < 3: continue
    im = transform_norm(temp/255.0)
    output, intermediates = model(im[None,], return_intermediates=True)
    gd24.append(intermediates[3][0,].detach().numpy())

gd24 = np.stack(gd24, axis=0)

In [7]:
f_list = glob.glob('good_batch/GD/*tif')
gd72 = []
for f in f_list:
    temp=io.imread(f, as_gray=False, plugin='tifffile').astype(np.float32)
    if temp.ndim < 3: continue
    im = transform_norm(temp/255.0)
    output, intermediates = model(im[None,], return_intermediates=True)
    gd72.append(intermediates[3][0,].detach().numpy())

gd72 = np.stack(gd72, axis=0)

In [8]:
f_list = glob.glob('3d gastruloids all images/WT/24H/*tif')
wt24 = []
for f in f_list:
    temp=io.imread(f, as_gray=False, plugin='tifffile').astype(np.float32)
    if temp.ndim < 3: continue
    im = transform_norm(temp/255.0)
    output, intermediates = model(im[None,], return_intermediates=True)
    wt24.append(intermediates[3][0,].detach().numpy())

wt24 = np.stack(wt24, axis=0)

In [9]:
f_list = glob.glob('good_batch/WT/*tif')
wt72 = []
for f in f_list:
    temp=io.imread(f, as_gray=False, plugin='tifffile').astype(np.float32)
    if temp.ndim < 3: continue
    im = transform_norm(temp/255.0)
    output, intermediates = model(im[None,], return_intermediates=True)
    wt72.append(intermediates[3][0,].detach().numpy())

wt72 = np.stack(wt72, axis=0)

In [10]:
# Combine
combined_im = np.vstack([mr24, mr72, gd24, gd72, wt24, wt72])
combined_tag = np.concatenate([np.full(mr24.shape[0], fill_value=0),
                               np.full(mr72.shape[0], fill_value=1),
                               np.full(gd24.shape[0], fill_value=2),
                               np.full(gd72.shape[0], fill_value=3),
                               np.full(wt24.shape[0], fill_value=4),
                               np.full(wt72.shape[0], fill_value=5)])

In [12]:
combined_im.shape

(434, 7, 7, 768)

In [13]:
## CNN Based
import tensorflow as tf

cnn_m = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                             tf.keras.layers.RandomRotation(0.7),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.GaussianNoise(0.1),
                             tf.keras.layers.SpatialDropout2D(0.5),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='gelu'),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='gelu'),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer=tf.keras.regularizers.L1(1e-3))])

cnn_m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy')

2025-01-04 11:14:23.163073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-04 11:14:25.494363: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-04 11:14:55.680594: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-01-04 11:14:55.680639: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on

In [16]:
one_hot = np.eye(6)[combined_tag,]
x_train, x_test, y_train, y_test = train_test_split(combined_im, one_hot, test_size=0.3, shuffle=True)
cnn_m.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=150, shuffle=True, batch_size=32)

Epoch 1/150
10/10 [==============================] - 4s 45ms/step - loss: 3.1677 - val_loss: 1.9252
Epoch 2/150
10/10 [==============================] - 0s 21ms/step - loss: 2.7748 - val_loss: 1.8436
Epoch 3/150
10/10 [==============================] - 0s 21ms/step - loss: 2.6191 - val_loss: 1.7692
Epoch 4/150
10/10 [==============================] - 0s 21ms/step - loss: 2.4415 - val_loss: 1.7025
Epoch 5/150
10/10 [==============================] - 0s 21ms/step - loss: 2.1903 - val_loss: 1.6470
Epoch 6/150
10/10 [==============================] - 0s 21ms/step - loss: 2.1211 - val_loss: 1.6047
Epoch 7/150
10/10 [==============================] - 0s 21ms/step - loss: 2.1842 - val_loss: 1.5586
Epoch 8/150
10/10 [==============================] - 0s 21ms/step - loss: 1.9806 - val_loss: 1.5169
Epoch 9/150
10/10 [==============================] - 0s 21ms/step - loss: 2.0145 - val_loss: 1.4761
Epoch 10/150
10/10 [==============================] - 0s 21ms/step - loss: 2.0383 - val_loss: 1.4487

In [19]:
emb_lay = cnn_m.layers[9](cnn_m.layers[8](cnn_m.layers[7](cnn_m.layers[6](cnn_m.layers[5](cnn_m.layers[4](cnn_m.layers[9](cnn_m.layers[3](cnn_m.layers[2](cnn_m.layers[1](cnn_m.layers[0](combined_im)))))))))))

In [21]:
# Do Isomap
pca_res = Isomap(n_neighbors=100).fit_transform(emb_lay)

col_key = ['red', 'firebrick', 'dodgerblue', 'navy', 'lime', 'forestgreen']
cust_leg = [Line2D([0], [0], color=s, lw=4) for s in col_key]


fig, ax = plt.subplots(1, 1)
ax.scatter(pca_res[::,0], pca_res[::,1], c=[col_key[i] for i in combined_tag], alpha=0.6)
ax.set_xlabel('Isomap 1')
ax.set_ylabel('Isomap 2')
ax.legend(cust_leg, ['MR 24', 'MR 72', 'GD 24', 'GD 72', 'WT 24', 'WT 72'], loc='upper left')
#plt.show()

plt.savefig('isomap_hiera.pdf')

In [22]:
tags = ['MR 24', 'MR 72', 'GD 24', 'GD 72', 'WT 24', 'WT 72']

pred_res = cnn_m.predict(x_test)
pred_res.shape
res_cat = [tags[i] for i in np.argmax(pred_res, axis=-1)]
res_test = [tags[i] for i in np.argmax(y_test, axis=-1)]

5/5 [==============================] - 0s 4ms/step


In [23]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cnf = confusion_matrix(res_test, res_cat, labels=tags)


fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.heatmap(cnf, annot=True, linecolor='white', linewidths=1, ax=ax, xticklabels=tags, yticklabels=tags)

plt.savefig('ConfusionMatrixCNN_K6_F30.pdf')

In [17]:
# Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

tags = ['MR 24', 'MR 72', 'GD 24', 'GD 72', 'WT 24', 'WT 72']

X=combined_im.reshape([434, 7*7*768])
y=[tags[i] for i in combined_tag]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = RandomForestClassifier(max_depth=3, n_jobs=4, class_weight='balanced', n_estimators=1500)

clf.fit(x_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=3, n_estimators=1500,
                       n_jobs=4)

In [18]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cnf = confusion_matrix(clf.predict(x_test), y_test)

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.heatmap(cnf, annot=True, linecolor='white', linewidths=1, ax=ax, xticklabels=tags, yticklabels=tags)

plt.savefig('ConfusionMatrixRF.pdf')

In [20]:
clf.score(x_test, y_test)

0.6091954022988506

In [21]:
## CNN Based
import tensorflow as tf

cnn_m = tf.keras.Sequential([tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.SpatialDropout2D(0.5),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='gelu'),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='gelu'),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer=tf.keras.regularizers.L1(1e-3))])

In [22]:
cnn_m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy')

In [23]:
from sklearn.preprocessing import LabelBinarizer

one_hot = LabelBinarizer().fit_transform(y)
one_hot

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [24]:
x_train, x_test, y_train, y_test = train_test_split(combined_im, one_hot, test_size=0.3, shuffle=True)

In [25]:
cnn_m.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=150, shuffle=True, batch_size=32)

Epoch 1/150
10/10 [==============================] - 10s 32ms/step - loss: 3.1148 - val_loss: 1.8846
Epoch 2/150
10/10 [==============================] - 0s 11ms/step - loss: 2.8698 - val_loss: 1.8238
Epoch 3/150
10/10 [==============================] - 0s 11ms/step - loss: 2.5273 - val_loss: 1.7709
Epoch 4/150
10/10 [==============================] - 0s 11ms/step - loss: 2.3967 - val_loss: 1.7183
Epoch 5/150
10/10 [==============================] - 0s 10ms/step - loss: 2.1999 - val_loss: 1.6657
Epoch 6/150
10/10 [==============================] - 0s 11ms/step - loss: 1.9912 - val_loss: 1.6199
Epoch 7/150
10/10 [==============================] - 0s 11ms/step - loss: 1.8921 - val_loss: 1.5760
Epoch 8/150
10/10 [==============================] - 0s 11ms/step - loss: 1.9957 - val_loss: 1.5382
Epoch 9/150
10/10 [==============================] - 0s 11ms/step - loss: 1.8748 - val_loss: 1.4924
Epoch 10/150
10/10 [==============================] - 0s 11ms/step - loss: 1.4637 - val_loss: 1.460

In [26]:
pred_res = cnn_m.predict(x_test)
pred_res.shape
res_cat = [tags[i] for i in np.argmax(pred_res, axis=-1)]
res_test = [tags[i] for i in np.argmax(y_test, axis=-1)]

5/5 [==============================] - 0s 2ms/step


In [27]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cnf = confusion_matrix(res_test, res_cat)


fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.heatmap(cnf, annot=True, linecolor='white', linewidths=1, ax=ax, xticklabels=tags, yticklabels=tags)

plt.savefig('ConfusionMatrixCNN.pdf')

In [28]:
cnn_m.layers

In [29]:
emb_lay = cnn_m.layers[5](cnn_m.layers[4](cnn_m.layers[3](cnn_m.layers[2](cnn_m.layers[1](cnn_m.layers[0](combined_im, training=False), training=False)))), training=False)

In [30]:
pca_res = Isomap(n_neighbors=30).fit_transform(emb_lay)

col_key = ['red', 'sandybrown', 'cyan', 'navy', 'darkviolet', 'forestgreen']
cust_leg = [Line2D([0], [0], color=s, lw=4) for s in col_key]


fig, ax = plt.subplots(1, 1)
ax.scatter(pca_res[::,0], pca_res[::,1], c=[col_key[i] for i in combined_tag], alpha=0.6)
ax.set_xlabel('Isomap 1')
ax.set_ylabel('Isomap 2')
ax.legend(cust_leg, ['MR 24', 'MR 72', 'GD 24', 'GD 72', 'WT 24', 'WT 72'], loc='upper left')
#plt.show()

plt.savefig('isomap_hiera_cnn.pdf')

In [153]:
np.sum(np.diag(cnf))/np.sum(cnf)

0.6641221374045801

In [4]:
def load_im(path=""):
    f_list = glob.glob(path)
    local_data = []
    for f in f_list:
        temp=io.imread(f, as_gray=False, plugin='tifffile').astype(np.float32)
        im = transform_norm(temp/255.0)
        output, intermediates = model(im[None,], return_intermediates=True)
        local_data.append(intermediates[3][0,].detach().numpy())

    return(np.stack(local_data, axis=0))

In [5]:
## AKT Inhibition
gd72_neg = np.vstack([load_im('gd neg/72*tif'), load_im('gd neg/gd neg*tif')])
gd72_pos = np.vstack([load_im('gd pos/72*tif'), load_im('gd pos/gd pos*tif')])

In [6]:
mr72_neg = np.vstack([load_im('mr neg/72*tif'), load_im('mr neg/mr neg*tif')])
mr72_pos = np.vstack([load_im('mr pos/72*tif'), load_im('mr pos/mr pos*tif')])

In [7]:
wt72_neg = np.vstack([load_im('wt neg/72*tif'), load_im('wt neg/wt neg*tif')])
wt72_pos = np.vstack([load_im('wt pos/72*tif'), load_im('wt pos/wt pos*tif')])

In [8]:
## CNN Based
import tensorflow as tf

cnn_m = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                             tf.keras.layers.RandomRotation(0.7),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.GaussianNoise(0.1),
                             tf.keras.layers.SpatialDropout2D(0.5),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='gelu'),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='gelu'),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer=tf.keras.regularizers.L1(1e-3))])

cnn_m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy')

2025-01-26 10:37:03.913028: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-26 10:37:07.827713: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-26 10:38:10.277961: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-01-26 10:38:10.278009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on

In [9]:
# Combine
combined_im = np.vstack([mr72_neg, mr72_pos, gd72_neg, gd72_pos, wt72_neg, wt72_pos])
combined_tag = np.concatenate([np.full(mr72_neg.shape[0], fill_value=0),
                               np.full(mr72_pos.shape[0], fill_value=1),
                               np.full(gd72_neg.shape[0], fill_value=2),
                               np.full(gd72_pos.shape[0], fill_value=3),
                               np.full(wt72_neg.shape[0], fill_value=4), 
                               np.full(wt72_pos.shape[0], fill_value=5)])

one_hot = np.eye(6)[combined_tag,]

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

tags = ['MR 72_Neg', 'MR 72_Pos',
       'GD 72_Neg', 'GD 72_Pos',
       'WT 72_Neg', 'WT 72_Pos']

#tags = ['Intermediate', 'Low', 'High']

y=[tags[i] for i in combined_tag]

In [73]:
x_train, x_test, y_train, y_test = train_test_split(combined_im, one_hot, test_size=0.3, shuffle=True)

In [74]:
cnn_m.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=150, shuffle=True, batch_size=16)

Epoch 1/150
21/21 [==============================] - 4s 23ms/step - loss: 3.1558 - val_loss: 2.0395
Epoch 2/150
21/21 [==============================] - 0s 14ms/step - loss: 3.1009 - val_loss: 1.9750
Epoch 3/150
21/21 [==============================] - 0s 14ms/step - loss: 2.6929 - val_loss: 1.9437
Epoch 4/150
21/21 [==============================] - 0s 14ms/step - loss: 2.8034 - val_loss: 1.9173
Epoch 5/150
21/21 [==============================] - 0s 14ms/step - loss: 2.5902 - val_loss: 1.9078
Epoch 6/150
21/21 [==============================] - 0s 14ms/step - loss: 2.6807 - val_loss: 1.8509
Epoch 7/150
21/21 [==============================] - 0s 14ms/step - loss: 2.6239 - val_loss: 1.8014
Epoch 8/150
21/21 [==============================] - 0s 14ms/step - loss: 2.6364 - val_loss: 1.7511
Epoch 9/150
21/21 [==============================] - 0s 14ms/step - loss: 2.5392 - val_loss: 1.7705
Epoch 10/150
21/21 [==============================] - 0s 14ms/step - loss: 2.5171 - val_loss: 1.7965

In [75]:
pred_res = cnn_m.predict(x_test)
pred_res.shape
res_cat = [tags[i] for i in np.argmax(pred_res, axis=-1)]
res_test = [tags[i] for i in np.argmax(y_test, axis=-1)]

5/5 [==============================] - 0s 4ms/step


In [76]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cnf = confusion_matrix(res_test, res_cat, labels=tags)


fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.heatmap(cnf, annot=True, linecolor='white', linewidths=1, ax=ax, xticklabels=tags, yticklabels=tags)

plt.savefig('ConfusionMatrixCNN_Akt_K6_F30.pdf')

In [77]:
cnn_m.save('cnn_model_akt_k6_f30.keras')

In [12]:
# Load model
cnn_m = tf.keras.models.load_model('cnn_model_akt.keras')
emb_lay = cnn_m.layers[9](cnn_m.layers[8](cnn_m.layers[7](cnn_m.layers[6](cnn_m.layers[5](cnn_m.layers[4](cnn_m.layers[9](cnn_m.layers[3](cnn_m.layers[2](cnn_m.layers[1](cnn_m.layers[0](combined_im)))))))))))

In [13]:
tags = ['MR 72_Neg', 'MR 72_Pos',
       'GD 72_Neg', 'GD 72_Pos',
       'WT 72_Neg', 'WT 72_Pos']

#tags = ['Intermediate', 'Low', 'High']

pca_res = Isomap(n_neighbors=100).fit_transform(emb_lay)

col_key = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
cust_leg = [Line2D([0], [0], color=s, lw=4) for s in col_key]


fig, ax = plt.subplots(1, 1)
ax.scatter(pca_res[::,0], pca_res[::,1], c=[col_key[i] for i in combined_tag], alpha=0.6)
ax.set_xlabel('Isomap 1')
ax.set_ylabel('Isomap 2')
ax.legend(cust_leg, tags, loc='upper left')
#plt.show()

plt.savefig('isomap_hiera_cnn_akt_model_k6.pdf')